In [4]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split


Epoch 1/10
3/3 [==============================] - 8s 2s/step - loss: 2.7066 - accuracy: 0.0909 - val_loss: 2.7082 - val_accuracy: 0.1364
Epoch 2/10
3/3 [==============================] - 7s 2s/step - loss: 2.6932 - accuracy: 0.2955 - val_loss: 2.7086 - val_accuracy: 0.0909
Epoch 3/10
3/3 [==============================] - 7s 2s/step - loss: 2.6793 - accuracy: 0.4545 - val_loss: 2.7101 - val_accuracy: 0.1364
Epoch 4/10
3/3 [==============================] - 7s 2s/step - loss: 2.6597 - accuracy: 0.3523 - val_loss: 2.7147 - val_accuracy: 0.0909
Epoch 5/10
3/3 [==============================] - 7s 2s/step - loss: 2.6245 - accuracy: 0.2045 - val_loss: 2.7592 - val_accuracy: 0.0909
Epoch 6/10
1/1 [==============================] - 1s 501ms/step - loss: 2.6939 - accuracy: 0.1071


In [ ]:

df = pd.read_csv('srpski.csv', sep='\t')

df = df.drop(columns=['Rbr', 'SR', 'sr/sr', 'Naslov', 'Jezik'])
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = df[column].str.lower()


In [ ]:

max_text_length = df['Tekst'].apply(lambda x: len(str(x).split())).max()

X_train, X_test, y_train, y_test = train_test_split(df['Tekst'], df['Autor'], test_size=0.2, random_state=42, stratify=df['Autor'])


In [ ]:


early_stopping = EarlyStopping(
    patience=3,  
    monitor='accuracy',  
    restore_best_weights=True  
)


In [ ]:


tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X = pad_sequences(sequences, maxlen=max_text_length)



In [ ]:

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=max_text_length))
model.add(LSTM(100))
model.add(Dense(len(set(df['Autor']))+1, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X, y_train, epochs=10, validation_split=0.2, callbacks=early_stopping)



In [ ]:

new_sequences = tokenizer.texts_to_sequences(X_test)
new_X = pad_sequences(new_sequences, maxlen=max_text_length)

predictions = model.predict(new_X)

evaluation_result = model.evaluate(new_X, y_test)

In [6]:
evaluation_result[1]

0.1071428582072258